<a href="https://colab.research.google.com/github/10Yash29/Plagiarism/blob/main/hackx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers torch

In [2]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in 

In [3]:
from transformers import BertTokenizer, BertModel
from transformers.utils import logging

logging.set_verbosity_warning()
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', use_auth_token=True)
model = BertModel.from_pretrained('bert-base-uncased', use_auth_token=True)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2135: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:3220: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [11]:
import os
import torch
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity

# Initialize BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
model.eval()

def get_embedding(text):
    # Tokenize the text
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)

    # Use the mean of the token embeddings for the text representation
    last_hidden_states = outputs.last_hidden_state
    embeddings = last_hidden_states.mean(dim=1)
    return embeddings

# Directory where the code submissions for a single question are stored
DATABASE_DIR = '/content/training data/original_1'
data = []
# Load code submissions and labels from text files
def load_training_data():
    if not os.path.exists(DATABASE_DIR):
        raise FileNotFoundError(f"The directory {DATABASE_DIR} does not exist.")

    data = []
    labels = []

    # Loop through all files in the directory
    for filename in os.listdir(DATABASE_DIR):
        file_path = os.path.join(DATABASE_DIR, filename)
        with open(file_path, 'r') as f:
            code = f.read().strip()

            if len(code) == 0:
                print(f"Skipping empty file: {filename}")
                continue

            data.append(code)

            if 'original' in filename:
                labels.append(0)  # Original
            else:
                labels.append(1)  # Plagiarized

    if len(data) == 0:
        raise ValueError("No valid code files found.")

    return data, labels

# Example usage to load data
try:
    data, labels = load_training_data()
except (FileNotFoundError, ValueError) as e:
    print(e)

if data:
    # Compute embeddings for training data
    train_embeddings = [get_embedding(code) for code in data]

    # Hardcoded test submission
    test_code = """
    class Solution:
        def searchMatrix(self, matrix: List[List[int]], target: int) -> bool:
            top_row = 0
            bottom_row = len(matrix) - 1
            while top_row <= bottom_row:
                middle_row = (top_row + bottom_row) // 2
                if matrix[middle_row][0] < target and matrix[middle_row][-1] > target:
                    break
                elif matrix[middle_row][0] > target:
                    bottom_row = middle_row - 1
                else:
                    top_row = middle_row + 1
            row_to_search = (top_row + bottom_row) // 2
            left_col = 0
            right_col = len(matrix[row_to_search]) - 1
            while left_col <= right_col:
                middle_col = (left_col + right_col) // 2
                if matrix[row_to_search][middle_col] == target:
                    return True
                elif matrix[row_to_search][middle_col] > target:
                    right_col = middle_col - 1
                else:
                    left_col = middle_col + 1
            return False
    """

    # Compute embedding for test code
    test_embedding = get_embedding(test_code)

    # Compute cosine similarities between test code and training data
    similarities = [cosine_similarity(test_embedding.numpy(), train_embedding.numpy())[0][0] for train_embedding in train_embeddings]

    # Check if the test code is plagiarized
    if max(similarities) > 0.8:
        print("The test submission is plagiarized.")
    else:
        print("The test submission is original.")


The test submission is plagiarized.
